Practical 6: Cuda Program for finding perfect Number

Steps:
1. Enable GPU in Colab

    Go to Runtime > Change runtime type

    Set Hardware Accelerator to GPU

   Click Save
2. !nvidia-smi    // GPU availability
3. !nvcc --version

4. Write your cuda file  eg: %%writefile test.cu followed by code

5. Compile your cuda program
    !nvcc -arch=sm_75 -o test test.cu

6. Run the code
    !./test




Program 1 :

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
%%writefile perfect_number.cu
#include <stdio.h>

__global__ void perfect() {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int num = tid + 1;

    if (num > 1000) return;

    int sum = 1;

    for (int p = 2; p <= num / 2; ++p) {
        if (num % p == 0) sum += p;
    }

    if (sum == num && num != 1) {
        printf("%d is a perfect number\n", num);
    }
}

int main() {
    int threadsPerBlock = 256;
    int totalNumbers = 1000;
    int numBlocks = (totalNumbers + threadsPerBlock - 1) / threadsPerBlock;

    perfect<<<numBlocks, threadsPerBlock>>>();
    cudaDeviceSynchronize();

    return 0;
}


Writing perfect_number.cu


In [3]:

!nvcc -arch=sm_75 -o perfect_number perfect_number.cu


In [4]:
!./perfect_number

6 is a perfect number
28 is a perfect number
496 is a perfect number


Program 3: finding perfect number using different cuda code

In [ ]:
%%writefile perfect_number1.cu
#include <stdio.h>

__device__ bool isPerfect(int num) {
    int sum = 1;
    for (int i = 2; i <= num / 2; i++) {
        if (num % i == 0) sum += i;
    }
    return sum == num;
}

__global__ void findPerfectNumbers(int *results, int *count, int limit) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int num = idx + 2;  // Start checking from 2

    if (num > limit) return;

    if (isPerfect(num)) {
        int pos = atomicAdd(count, 1);
        results[pos] = num;
    }
}

int main() {
    const int LIMIT = 10000;
    const int MAX_RESULTS = 10;

    int *d_results, *d_count;
    int h_results[MAX_RESULTS] = {0};
    int h_count = 0;

    cudaMalloc(&d_results, MAX_RESULTS * sizeof(int));
    cudaMalloc(&d_count, sizeof(int));

    cudaMemset(d_results, 0, MAX_RESULTS * sizeof(int));
    cudaMemset(d_count, 0, sizeof(int));

    int threadsPerBlock = 256;
    int totalThreads = LIMIT - 1; // from 2 to LIMIT
    int numBlocks = (totalThreads + threadsPerBlock - 1) / threadsPerBlock;

    findPerfectNumbers<<<numBlocks, threadsPerBlock>>>(d_results, d_count, LIMIT);
    cudaDeviceSynchronize();

    cudaMemcpy(&h_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_results, d_results, MAX_RESULTS * sizeof(int), cudaMemcpyDeviceToHost);

    printf("Perfect numbers up to %d:\n", LIMIT);
    if (h_count == 0) {
        printf("No perfect numbers found.\n");
    } else {
        for (int i = 0; i < h_count; ++i) {
            printf("%d\n", h_results[i]);
        }
    }

    cudaFree(d_results);
    cudaFree(d_count);

    return 0;
}


Writing perfect_number1.cu


In [ ]:
!nvcc -arch=sm_75 -o testt perfect_number1.cu

In [ ]:
!./testt

Perfect numbers up to 10000:
6
28
496
8128
